What makes a good recepie?
first we need to import the revelevant libraries to start scraping

In [1]:
import matplotlib.pyplot as plt
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup  
import pandas as pd
import scipy as sc
import numpy as np
import requests
import re
import time
import csv



now we want to start scraping we will first get all the receipie names and links and save them to a file

In [2]:
def get_full_page_thespruceeats():#this returns a list of all the links to receipies on the page:
    # URL to scrape
    url = "https://www.thespruceeats.com/search?q=&searchType=recipe"

    # Configure the Selenium webdriver
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')  # Run in headless mode (no GUI)
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    # Wait for the page to load
    wait = WebDriverWait(driver, 10)

    # Get the page source and parse it with BeautifulSoup
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "html.parser")

    results_div = soup.find("div", attrs={"class": "results-list__container"})
    recipe_names = []
    recipe_links = []

    # Scrape the first page
    for li in results_div.find_all("li", class_="results__item"):
        if li.find("a") is not None:
            link = li.find("a").get("href")
        else:
            link = ''

        if li.find("h4", class_="card__title") is not None:
            name = li.find("h4", class_="card__title").text.strip()
        else:
            name = ''
        
        recipe_names.append(name)
        recipe_links.append(link)

    # Scrape subsequent pages if the "Next" button exists
    while True:
        try:
            next_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".pagination__item-link--next")))
            next_button.click()
            time.sleep(5)

            # Get the page source and parse it with BeautifulSoup
            page_source = driver.page_source
            soup = BeautifulSoup(page_source, "html.parser")

            results_div = soup.find("div", attrs={"class": "results-list__container"})

            # Scrape recipe names and links
            for li in results_div.find_all("li", class_="results__item"):
                if li.find("a") is not None:
                    link = li.find("a").get("href")
                else:
                    link = ''

                if li.find("h4", class_="card__title") is not None:
                    name = li.find("h4", class_="card__title").text.strip()
                else:
                    name = ''
                print(f"{name} added")
                recipe_names.append(name)
                recipe_links.append(link)

        except:
            break

    # Create a DataFrame with the recipe names and links
    df = pd.DataFrame({"Recipe_name": recipe_names, "Recipe_link": recipe_links})

    # Write DataFrame to a CSV file
    df.to_csv("Recipe_Links_and_Names.csv", index=False)

    # Close the driver
    driver.quit()

    print("Done!")

lets run this function!. we save this to a csv to speed up work later as we are no longer depenadant on the webdriver in another place and also in case of internet lose during the next tests

In [3]:
get_full_page_thespruceeats()

Pepper Steak Stir-Fry Recipe added
Grilled Salmon Burgers With Radicchio Slaw and Sambal Mayonnaise added
Kentucky Buck Cocktail Recipe added
Sparkling Borage Cocktail added
Lunch Box-Worthy Falafel Kebabs added
Chipotle Pumpkin Queso Dip added
Flamin' Hot Cheetos Mac and Cheese Bites added
Cherry Vinegar Recipe added
Turkish Ramadan Flat Bread (Pide) added
Carrot, Cabbage, and Kohlrabi Slaw With Miso Dressing Recipe added
Esquites Recipe (Mexican Corn Off the Cob) added
Gillie Fix Cocktail Recipe added
Raspberry Snakebite Recipe added
Stuffed Italian Sourdough Loaf Recipe added
Strawberry Chicken Salad With Champagne Vinaigrette Recipe added
Lasagne All'astice (Lobster Lasagna) Recipe added
Pear and Pomegranate Champagne Shrub Recipe added
Homemade Smoked Maple Bacon added
Baked S’mores Skillet Dip Recipe added
Garlic Chicken Primavera Pasta added
Cold Soba Noodle Salad Recipe added
Vegetarian Tofu Tacos added
Bay Hill Hummer added
A Recipe for Risotto Made With Amarone Wine (Risotto 

now that we have the receipie links we need to scrape each one we will break this into steps as much as possible first we need to soup object

In [ ]:
def load_soup_object(url):
    ###
    #url = "https://www.thespruceeats.com/search?q=&searchType=recipe"
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    return soup
    ###

next we will gather information by the order it is represented in our site using functions because we will loop over it soon!

our first function will gather the cook time prep time total time to cook and the amount of servings
this was very messy because of the formating can change widly from one recepie to another time could look like 25 min or 2h 15 min or 2 hours 10 min or countless other variations that why we used a complex regex expression to combat this issue 

In [ ]:
def get_cook_times(soup_obj):
    lines=[]
    results_items = soup_obj
    results_items = soup_obj.find_all(class_='comp article__decision-block mntl-block')
    if(results_items==[]):
        soup = soup_obj
        results_items = soup.find_all(class_='comp project-meta')        

    for item in results_items:
        item.find_all(class_='meta-text__data')
        for sub_item in item:
            if bool(sub_item.text.strip()):
                clean_text = sub_item.text.strip().replace('\n', '')
                lines.append(clean_text)

    if(len(lines)>1):
        new_string = lines[0] + lines[1]
        lines[0]= new_string

    #use regex expressions to clean up the line we get it looks something like this
    #['Prep: 15 minsCook: 20 minsTotal: 35 minsServings: 6 servingsYield: 1 cake', 'ratingsAdd a comment']
    #prep = re.findall(r'Prep:\s*(\d+)\s*mins', lines[0])[0]
    cook_time_str = re.findall(r'Cook:\s*(?:(\d+)\s*(?:hrs?|hours?)\s*)?(?:(\d+)\s*mins?)?', lines[0])[0]
    prep_time_str = re.findall(r'Prep:\s*(?:(\d+)\s*(?:hrs?|hours?)\s*)?(?:(\d+)\s*mins?)?', lines[0])[0]    
    total_time_str = re.findall(r'Total:\s*(?:(\d+)\s*(?:hrs?|hours?)\s*)?(?:(\d+)\s*mins?)?', lines[0])[0]
# Convert cook time to minutes


    hours = int(cook_time_str[0]) if cook_time_str[0] else 0
    minutes = int(cook_time_str[1]) if cook_time_str[1] else 0
    cook_time_minutes = hours * 60 + minutes

    hours = int(prep_time_str[0]) if prep_time_str[0] else 0
    minutes = int(prep_time_str[1]) if prep_time_str[1] else 0
    prep_time_minutes = hours * 60 + minutes


    hours = int(total_time_str[0]) if total_time_str[0] else 0
    minutes = int(total_time_str[1]) if total_time_str[1] else 0
    total_minutes = hours * 60 + minutes
    #total = re.findall(r'Total:\s*(\d+)\s*mins', lines[0])[0]
    #servings = re.findall(r'Servings?:\s*(\d+?)\s*(?:to\s*\d+)?\s*(?:servings|ratings)', lines[0])[0] # sometimes instead of saying servings 6 they say servings 6 to 8 in this case we make it servings 6
    #servings = re.findall(r'servings?:\s*(\d+?)\s*(?:to\s*\d+)?\s*(?:servings?|ratings)', lines[0], re.IGNORECASE)[0]
    #text = "The serving size is 3 servings per container."
    
    if(lines[0].count('serv')):
        match = re.search(r'serv\w*:\D*(\d+)', lines[0], re.IGNORECASE)
        if match:
            servings=(match.group(1))
    else:
        servings=1

    

    # Create a pandas DataFrame from the extracted data
    df = pd.DataFrame({
        'Prep': [prep_time_minutes],
        'Cook': [cook_time_minutes],
        'Total': [total_minutes],
        'Servings': [servings]
    })
    df = df.astype(int)
    return df

now for the second block of information which contains the rating for the dish 
this one was also messy because instead of writing a number they only display the star rating with 0.5 increments so we needed to count how many full stars and half stars there are 

In [ ]:
def get_stars(soup_obj):    
    soup = soup_obj
    results_items = soup.find_all(class_='comp js-feedback-trigger aggregate-star-rating mntl-block')    
    #print(results_items.prettify())
    for item in results_items:##result items size is 1
        text=item.prettify()
        full_stars=text.count('class="active"')
        half_stars=text.count('class="half"')
        return(full_stars+0.5*half_stars)

up next is the rating count nothing to special about this one 

In [ ]:
def get_rating_count(soup_obj):
    soup = soup_obj
    rating_elements = soup.find_all("div", attrs={'class': "comp aggregate-star-rating__count mntl-aggregate-rating mntl-text-block"})
    for rating_element in rating_elements:
        rating_text = rating_element.text.strip()
        try:
            num_ratings = int(rating_text.split()[0])
            return num_ratings
        except ValueError:
            pass
    return 0

up next we have the 3rd block of information which gives us the nutritional values there was an issue here that alcoholic beverages had no nutrinal value and the site had a lot of those

In [ ]:
def get_nutritional_values(soup_obj):
    soup = soup_obj
    results_items = soup.find_all(class_='nutrition-info__table--row')

    nutritional_vals=[]    


    for item in results_items:
        nutritional_vals.append(item.text.strip())
    new_list = []
    for s in nutritional_vals:
        # Split the string by the \n character and add the two parts to a new list
        parts = s.split('\n')
        # Add the new strings to the new list in the desired format
        #[caleories:934,fat:134g,carbs:999]
        new_list.extend([parts[1], parts[0]])
    #[calories,934,far,134g,carbs,1123,]
    df = pd.DataFrame({'nutrient': new_list[::2], 'value': new_list[1::2]})

    # Set 'nutrient' column as index and transpose DataFrame
    df = df.set_index('nutrient').T
    if df.empty: #recepies like cocktails have no calories

        df = pd.DataFrame(columns=['nutrient', 'Calories', 'Fat', 'Carbs', 'Protein'])

        # add a row filled with zeros
        df.loc[0] = ['value', 0, '0g', '0g', '0g']
        df['Calories'] = df['Calories'].astype(np.int32)
        return df 

    df['Calories'] = df['Calories'].astype(int)

    return df

next is the 4th block of relevant information which holds the list of ingridients this was also not fun as the class names for some receipies changed and werent consistent resulting in a lot of hotfixes and hours of debugging which is why we added cond to check if its the old varient of the site or not and if its the new one we can sometimes filter the ingridients from the start!

In [ ]:
def get_ingridients(soup_obj):
    cond=0
    soup = soup_obj
    span_elements = soup.find_all('span', {'data-ingredient-name': 'true'})

    # create an empty list to store the ingredient names
    ingredient_names = []

    # loop over the span elements and extract their text content
    for span in span_elements:
        ingredient_names.append(span.text)
    #print(ingredient_names)
    if(len(ingredient_names)>0):
        return(ingredient_names)
    else:
        cond=0
        soup = soup_obj
        results_items = soup.find_all(class_='structured-ingredients__list text-passage')
                                            #comp ingredient-list simple-list simple-list--bulleted  
        #print(results_items)                                           
        if(results_items==[]): #sometimes they like to change the class name
            soup = soup_obj
            results_items = soup.find_all(class_='simple-list__item js-checkbox-trigger ingredient text-passage')
            cond=1
        nutritional_vals=[]
        if(results_items==[]):
            return []
        
        final_lst=[]

        for item in results_items:    
            nutritional_vals.append(item.text.strip())
            #print(item.text.strip())
        if(cond==1):
            return nutritional_vals
        else:        
            for i in nutritional_vals:
                my_list = [s.strip() for s in i.split('\n\n\n')]
                final_lst.extend(my_list)
                #print(final_lst)
            
            return(final_lst)

we also wanted to add some info of our own so we looked at the ingridients and tried to figure out if its dairy meat fur(parve) or dairy and meat as for keywords we just googled common meat or common dairy products and coppied the first few 

In [ ]:
def analyze_recipe(ingredients):
    dairy_keywords = ["milk", "cheese", "yogurt", "cream", "butter", "whey", "casein", "curds"]
    meat_keywords = ["beef", "chicken", "pork", "lamb", "turkey", "venison", "duck", "bacon", "sausage",
                     "ham", "prosciutto", "pepperoni", "salami", "chorizo", "bresaola", "pastrami",
                     "corned beef", "veal", "goose", "game", "elk", "bison", "rabbit", "boar", "guinea fowl", "quail"]
    
    categories = {'Dairy': 0, 'Meat': 0, 'Fur': 1}
    
    for ingredient in ingredients:
        ingredient = ingredient.lower()
        if any(keyword in ingredient for keyword in dairy_keywords):
            categories['Dairy'] = 1
            categories['Fur'] = 0
        elif any(keyword in ingredient for keyword in meat_keywords):
            categories['Meat'] = 1
            categories['Fur'] = 0
            
    df = pd.DataFrame(categories, index=[0])
    return df

now we want to merge all of this data into a single row of a dataframe and again we will be looping over this one 

In [ ]:
def merge_fast(url,recepie_name):
    df = pd.DataFrame(columns=['Name','Prep', 'Cook', 'Total', 'Servings', 'Rating','Rating_Count','Dairy','Meat','Fur', 'Calories', 'Fat', 'Carbs', 'Protein', 'Ingredients'])
    #Dairy
    #Meat
    #Fur
    soup_obj= load_soup_object(url)

    recipe_df = get_cook_times(soup_obj)

    ratings_list = get_stars(soup_obj)

    nutrition_df = get_nutritional_values(soup_obj)

    ingredients = get_ingridients(soup_obj)

    rating_num = get_rating_count(soup_obj)

    meatdairy_df = analyze_recipe(ingredients)

    if(ingredients==[]):
        ingredients=['','','','']
        print(type(nutrition_df['Calories'][0]))
    new_row = {
        'Name':recepie_name,
        'Prep': recipe_df['Prep'][0],
        'Cook': recipe_df['Cook'][0],
        'Total': recipe_df['Total'][0],
        'Servings': recipe_df['Servings'][0],
        'Rating': ratings_list,
        'Rating_Count':rating_num,
        'Dairy':meatdairy_df['Dairy'][0],
        'Meat':meatdairy_df['Meat'][0],
        'Fur':meatdairy_df['Fur'][0],
        'Calories': nutrition_df['Calories'][0],
        'Fat': nutrition_df['Fat'][0],
        'Carbs': nutrition_df['Carbs'][0],        
        'Protein': nutrition_df['Protein'][0],
        'Ingredients': [ingredients]
    }
    #print(new_row)

    # add the new row to the DataFrame
    df = pd.concat([df, pd.DataFrame(new_row)], ignore_index=True)
    #df_concat = pd.concat([df1, df2], keys=['df2'])
    df['Prep'] = df['Prep'].astype(float)
    df['Cook'] = df['Cook'].astype(float)
    df['Total'] = df['Total'].astype(float)
    df['Servings'] = df['Servings'].astype(float)
    df['Rating'] = df['Rating'    ].astype(float)
    df['Calories'] = df['Calories'].astype(float)

    # display the updated DataFrame
    #print("another line added")
    return(df)

and now we want to do this for each receipe that was found up until now here comes another function :)
it will take a while so again to speed up work later we are saving it to a file 

In [ ]:
def fast_scrape(csv_file_name):
    recipe_names = []
    recipe_links = []
    count=0

    with open(csv_file_name, encoding='utf-8', newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            recipe = ','.join(row).strip().replace('\x9c', '')  # Join recipe name and link with a comma, remove problematic characters
            last_comma = recipe.rfind(',')  # Find the index of the last comma in the recipe string
            if last_comma != -1:
                recipe_name = recipe[:last_comma].strip()  # Get the recipe name before the last comma
                recipe_link = recipe[last_comma+1:].strip()  # Get the recipe link after the last comma
                recipe_names.append(recipe_name)
                recipe_links.append(recipe_link)
            else:
                print(f"Invalid row: {row}")

    final_df=pd.DataFrame()
    for name, link in zip(recipe_names, recipe_links):
        # print(f"Recipe name: {name}")
        # print(f"Recipe link: {link}")
        temp_df=merge_fast(link,name)
        temp_df.set_index('Name', inplace=True)  # set the index of temp_df to the recipe name
        if(final_df.empty):
            final_df=temp_df
        else:
            final_df = pd.concat([final_df,temp_df])
        print(final_df)

    #final_df.to_pickle('dataframe.pkl')
    #final_df.to_pickle('dataframe.pkl', protocol=4, encoding='utf-8')
    final_df.to_csv('my_data.csv', index=True, encoding='utf-8')
    #print(final_df)
    return final_df


In [ ]:
df = fast_scrape('Recipe_Links_and_Names.csv')

MissingSchema: Invalid URL 'Recipe_link': No scheme supplied. Perhaps you meant http://Recipe_link?

Now that we have our dataframe lets see what we are working with. We will try to visualize the data and clean it up later of course we want some functions :)  We will make functions for scatter plot, 
histogram and a 1 column pie.

In [ ]:
def draw_scatter_2_params(col_name_1,col_name_2):
    df=pd.read_csv('my_data.csv')
    df['Fat'] = df['Fat'].str.replace('g', '').str.replace(',', '').astype(int)
    df['Carbs'] = df['Carbs'].str.replace('g', '').str.replace(',', '').astype(int)
    df['Protein'] = df['Protein'].str.replace('g', '').str.replace(',', '').astype(int)
    df.plot.scatter(x=col_name_1, y=col_name_2)
    
    plt.show()

In [ ]:
def draw_histo_1_params(col_name):
    # read in your dataframe from a csv file
    df = pd.read_csv('my_data.csv')
    df['Fat'] = df['Fat'].str.replace('g', '').str.replace(',', '').astype(int)
    df['Carbs'] = df['Carbs'].str.replace('g', '').str.replace(',', '').astype(int)
    df['Protein'] = df['Protein'].str.replace('g', '').str.replace(',', '').astype(int)
    # choose the column you want to use for the histogram

    # sort the column values into bins
    bin_values, bin_edges = np.histogram(df[col_name], bins='auto')

    # create the histogram using the sorted bins
    plt.hist(df[col_name], bins=bin_edges)

    # add labels and title to the histogram
    plt.xlabel('Values')
    plt.ylabel('Frequency')
    plt.title('Histogram of ' + col_name)

    # display the histogram
    plt.show()

In [ ]:
def draw_pie_1_params(col_name):
    df = pd.read_csv('my_data.csv')
    df['Fat'] = df['Fat'].str.replace('g', '').str.replace(',', '').astype(int)
    df['Carbs'] = df['Carbs'].str.replace('g', '').str.replace(',', '').astype(int)
    df['Protein'] = df['Protein'].str.replace('g', '').str.replace(',', '').astype(int)
    # choose the column you want to use for the pie chart    

    # get the count of unique values in the column
    value_counts = df[col_name].value_counts()

    # create the pie chart
    plt.pie(value_counts.values, labels=value_counts.index, autopct='%1.1f%%')

    # add title to the pie chart
    plt.title('Pie Chart of ' + col_name)

    # display the pie chart
    plt.show()

in addition we wanted a pie chart of how much is meat dairy meat and dairy and fur

In [ ]:
def draw_pie_meat_dairy_fur():
    df = pd.read_csv('my_data.csv')
    
    # Calculate the number of recipes in each category
    meat_count = len(df[df['Meat'] == 1])
    dairy_count = len(df[df['Dairy'] == 1])
    fur_count = len(df[df['Fur'] == 1])
    dairy_meat_count = len(df[(df['Dairy'] == 1) & (df['Meat'] == 1)])

    # Create a list of category counts and labels
    counts = [meat_count, dairy_count, fur_count, dairy_meat_count]
    labels = ['Meat', 'Dairy', 'Fur', 'Dairy&Meat']

    # Create the pie chart
    plt.pie(counts, labels=labels, autopct='%1.1f%%')

    # Add a title to the chart
    plt.title('Recipe Categories')

    # Show the chart
    plt.show()

lets run all of these and see if we can already draw some conclusions here come some more functions

In [ ]:
def draw_all_histo(df):
    #df=scraping_functions.clean_df('my_data.csv')
    # scraping_functions.save_df(df,'clean.csv')
    # df= pd.read_csv('clean.csv')
    numeric_cols = df.select_dtypes(include=['int', 'float']).columns.to_list()
    # print(numeric_cols)
    # numeric_cols.remove("Unnamed: 0")
    # print(numeric_cols)
    #print(numeric_cols)
    # # loop over the selected columns
    print(df.describe())
    for col in numeric_cols:
        draw_histo_1_params(df,col)

In [ ]:
draw_all_histo(df)

as we can see in some of those graphs(we need to add their names here)we have outliers. We need to clean up the data instead of going over each column and manually cleaning it we will clean the whole dataframe at once using the IQR methoed and some carefull tinkering and some columns have unnecesery letters that if we delete they turn into numeric columns(fat,carbs ...)

In [ ]:
def clean(df,column_name):
    
    print(df[column_name].describe())
    # Calculate the IQR of the Prep column
    Q1 = df[column_name].quantile(0.25)
    Q3 = df[column_name].quantile(0.75)
    IQR = Q3 - Q1

    # Determine the upper and lower bounds for the Prep column
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 10 * IQR

    # Replace any values outside of the bounds with NaN
    df.loc[(df[column_name] < lower_bound) | (df[column_name] > upper_bound), column_name] = float('NaN')
    #df = df.dropna(subset=[column_name])
    df.dropna(inplace=True)
    print(df[column_name].describe())
    return df

In [ ]:
def remove_g_fron_col(df,col_name):
    df[col_name] = df[col_name].str.replace('g', '').str.replace(',', '').astype(int)
    return df

In [ ]:
def clean_df(df):
    #df=pd.read_csv(filename)
    #df=clean_data_time(df)
    df=remove_g_fron_col(df,'Fat')
    df=remove_g_fron_col(df,'Carbs')
    df=remove_g_fron_col(df,'Protein')

    
    #print(df)
    numeric_cols = df.select_dtypes(include=['int', 'float']).columns
    #print(numeric_cols)
    # # loop over the selected columns
    for col in numeric_cols:
        clean(df,col)

    return df

In [ ]:
df = clean_df(df)

we also needed to fix some errors with the time parameters 

In [ ]:
def clean_data_time(df):
    #df = pd.read_csv('clean_df.csv')

    # Replace Prep, Cook, and Total values with median if Prep + Cook != Total
    for index, row in df.iterrows():
        if row['Prep'] + row['Cook'] != row['Total']:
            if row['Prep'] != 0:
                df.at[index, 'Cook'] = df.at[index, 'Total'] - df.at[index, 'Prep']
            elif row['Cook'] != 0:
                df.at[index, 'Prep'] = df.at[index, 'Total'] - df.at[index, 'Cook']
            else:
                df.drop(index, inplace=True)

    # Fill missing Total values with Prep + Cook
    df['Total'].fillna(df['Prep'] + df['Cook'], inplace=True)

    # Fill missing Prep or Cook values if only one is missing
    for index, row in df.iterrows():
        if pd.isna(row['Prep']):
            if pd.notna(row['Cook']) and pd.notna(row['Total']):
                df.at[index, 'Prep'] = df.at[index, 'Total'] - df.at[index, 'Cook']
            else:
                df.drop(index, inplace=True)
        elif pd.isna(row['Cook']):
            if pd.notna(row['Prep']) and pd.notna(row['Total']):
                df.at[index, 'Cook'] = df.at[index, 'Total'] - df.at[index, 'Prep']
            else:
                df.drop(index, inplace=True)
        elif pd.isna(row['Total']):
            if pd.notna(row['Prep']) and pd.notna(row['Cook']):
                df.at[index, 'Total'] = df.at[index, 'Prep'] + df.at[index, 'Cook']
            else:
                df.drop(index, inplace=True)

    # Save cleaned dataframe to new csv file
    #df = df.drop(columns='Unnamed')
    #df = df.drop(df.columns[0], axis=1)
    #print(df.columns)
    #df.to_csv('Cleaned_df_updated.csv', index=False)
    return df

In [ ]:
df = clean_data_time(df)

lets review the results and ensure we are ok with them (shpira plz fix)

In [ ]:
draw_all_histo(df)

ok looks good now lets try and visualize the data some more to try and "get a feel" for our data(shapira plz fix)

In [ ]:
def draw_pie_meat_dairy_fur(df):

    #df = pd.read_csv('my_data.csv')
    
    # Calculate the number of recipes in each category
    meat_count = len(df[df['Meat'] == 1])
    dairy_count = len(df[df['Dairy'] == 1])
    fur_count = len(df[df['Fur'] == 1])
    dairy_meat_count = len(df[(df['Dairy'] == 1) & (df['Meat'] == 1)])

    # Create a list of category counts and labels
    counts = [meat_count, dairy_count, fur_count, dairy_meat_count]
    labels = ['Meat', 'Dairy', 'Fur', 'Dairy&Meat']

    # Create the pie chart
    plt.pie(counts, labels=labels, autopct='%1.1f%%')

    # Add a title to the chart
    plt.title('Recipe Categories')

    # Show the chart
    plt.show()

In [ ]:
def draw_scatter_2_params(df,col_name_1,col_name_2):    
    # df['Fat'] = df['Fat'].str.replace('g', '').str.replace(',', '').astype(int)
    # df['Carbs'] = df['Carbs'].str.replace('g', '').str.replace(',', '').astype(int)
    # df['Protein'] = df['Protein'].str.replace('g', '').str.replace(',', '').astype(int)
    df.plot.scatter(x=col_name_1, y=col_name_2)
    
    plt.show()

we went over every possible option to draw a scatter plot and we found it hard to draw to correlations this way so we went with a mathematical aproach

In [ ]:
def get_correlation(df, col1, col2):
    return df[col1].corr(df[col2])

we ran this with each and every column 

In [ ]:
from statistics import correlation


def get_corr_all_columns(df):
    df = pd.read_csv('clean_modified.csv')
    numeric_cols = df.select_dtypes(include=['int', 'float'])
    if 'Unnamed' in numeric_cols.columns:
        numeric_cols = numeric_cols.drop('Unnamed', axis=1)

    correlation = {}
    for col1 in numeric_cols:
        for col2 in numeric_cols:
            if col1 != col2:
                corr_value = get_correlation(df, col1, col2)
                if corr_value > 0.5 or corr_value < -0.5:
                    correlation[(col1, col2)] = corr_value

    # Sort the correlations by their absolute value, in descending order
    sorted_correlations = sorted(correlation.items(), key=lambda x: abs(x[1]), reverse=True)

    # Print the correlations above 0.5 or below -0.5
    for corr, value in sorted_correlations:
        if abs(value) > 0.5:
            print(f"{corr[0]} and {corr[1]}: {value}")

sadly no correlation with rating 

our idea from here was to try and figure What factors contribute to the popularity of a recipe, and how accurately can we predict a recipe's popularity based on these factors? to that end we wanted to filter our ingridients as we assumed that particular ingridients could possitively or negetively effect the rating.
let the cleaning beggin!

lets see what we are working with 

In [ ]:
def initial_data_review(df):
    #Create a list of all the ingredients
    #df=pd.read_csv('my_data.csv')
    all_ingredients = []

    for i in df['Ingredients']:
        all_ingredients += eval(i)
    # Define a list of terms to exclude from the ingredients list
    exclude_terms = ['salt', 'pepper', 'garlic', 'onion', 'paprika', 'cumin', 'chili', 'oregano', 'basil', 'thyme', 'rosemary']
    # Create a list of all the ingredients
    clean_ingredients = []
    for ingredient in all_ingredients:
        ingredient = re.sub(r'\d+(\.\d+)?', '', ingredient) # Remove any quantity
        ingredient = re.sub(r'(\s+\d+)?\s*(large|medium|small)?\s*(cup|teaspoon|tablespoon)s?', '', ingredient, flags=re.IGNORECASE) # Remove any volume/capacity description
        ingredient = ingredient.strip()
        if ingredient and not any(term in ingredient.lower() for term in exclude_terms):
            clean_ingredients.append(ingredient)

    # Count the frequency of each ingredient
    ingredient_counts = pd.Series(clean_ingredients).value_counts()

    # Create a pie chart for the top 10 ingredients
    top_10_ingredients = ingredient_counts.head(10)
    plt.pie(top_10_ingredients, labels=top_10_ingredients.index, autopct='%1.1f%%')
    plt.title('Top 10 Ingredients in Recipes (Excluding Spices)')
    plt.show()

In [ ]:
initial_data_review(df)

as we can see the results dont realy teach us anything but we wanted to clean them because they come with data such as the volume or weight or special characters and that isn't what we need also the names differ when talking about the same thing like special salts they are all just salt it also results in some duplicates in some cases we need to fix this  

our idea was to make a dictionary of the top X ingridients and manully filter the junk out of them first lets make the dictionary 

In [ ]:
import collections


def find_most_common_ingridients(df):
    final_lst = []
    #df = pd.read_csv('clean.csv')
    ingredients = df['Ingredients'].tolist()

    for item in ingredients:
        temp = item.replace("'", "")   
        temp = item.replace("\\\\xa0", " ")
        temp = item.replace("\\\\u200b", " ")
        temp = temp.strip("[]").split(", ")
        temp_tokens = []
        for item in temp:
            temp_tokens.extend(item.split())
        final_lst.extend(temp_tokens)

    #print(final_lst)
    word_count = collections.Counter(final_lst)

    # Filter out items that start with a digit
    sorted_dict = {k: v for k, v in word_count.items() if not k[0].isdigit()}
    sorted_dict = dict(sorted(sorted_dict.items(), key=lambda x: x[1], reverse=True))
    print('\n\n\n\n')
    # Print the top 100 items

    # Create a DataFrame from the sorted dictionary and save it as a CSV file
    df = pd.DataFrame.from_dict(sorted_dict, orient='index', columns=['count'])
    df.to_csv('Ingredients_dirty.csv')
    
    for i, (key, value) in enumerate(sorted_dict.items()):
        if i == 300:
            break
        print(f"{key}: {value}")
    return sorted_dict

now with the power of notepad++ and some regex expressions we first made everything lower case delete any cell with numbers in it removed duplicates and clustered when it made sense 

(figure out how to upload a csv with the notebook we need to upload the ingridients)

now we have what our *relevant* ingridients should look like time to filter them out if an ingridient apears as a substring in the original ingridients swap it to what is in the new csv 

In [ ]:
def read_ingridients():
    with open('ingridients.csv', 'r') as file:
    # Initialize an empty list to store the data
        data = []
        for line in file:
            line = line.strip()
            values = line.split(',')
            return values
    return data

In [ ]:
def delete_index_columns(df):
    # Get a list of all column names that start with 'Unnamed'
    index_cols = [col for col in df.columns if col.startswith('Unnamed')]
    
    # Drop the columns from the dataframe
    df.drop(columns=index_cols, inplace=True)
    
    return df

In [ ]:
def clean_ingridients(df):
    df=delete_index_columns(df)
    df2=df.copy()
    pure_ingredients=read_ingridients()
    print("starting ingridient cleanup this might take a minute")
    #df=pd.read_csv(filename)
    for i, ingredient in enumerate(df2['Ingredients']):
        line = ingredient.split(",")
        for j, item in enumerate(line):
            for single_ingredient in pure_ingredients:
                if single_ingredient.lower() in item.lower():
                    #print(f'found {single_ingredient} in {item}')
                    line[j] = single_ingredient.lower()
        df2.loc[i, 'Ingredients'] = ",".join(line)
        #print(line)
        #print('\n\n')
    #print(df['Ingredients'])
    # save the modified dataframe to a new CSV file
    df2=delete_index_columns(df)
    #df2.to_csv('clean_modified.csv', index=False)
    print("done!")
   
    return df2

In [ ]:
df = clean_ingridients(df)

lets visualise this and hope that we cleaned it properly

In [ ]:
def get_ingridients_dict(df):
    ingredients = []

    with open('ingridients.csv', newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            ingredients.extend(row)

    #df= pd.read_csv('clean_modified.csv')


    ingredients_dict = {}

    # loop through the ingredients in the DataFrame
    for line in df['Ingredients']:
        for single_item in line.split(','): # split the ingredients by comma if they're in a single string
            for ing_in_lst in ingredients: # loop through the ingredients in the ingredients list
                if ing_in_lst in single_item.strip(): # check if the ingredient in the list is a substring of the ingredient in the dataset
                    # if the ingredient exists in the dictionary, increment its count
                    if ing_in_lst in ingredients_dict:
                        ingredients_dict[ing_in_lst] += 1
                    # otherwise, add the ingredient to the dictionary with a count of 1
                    else:
                        ingredients_dict[ing_in_lst] = 1

    # sort the dictionary by value in descending order
    sorted_dict = dict(sorted(ingredients_dict.items(), key=lambda x: x[1], reverse=True))

    # print the dictionary of ingredients and their counts
    # print("Ingredient Counts:")
    # for ing, count in sorted_dict.items():
    #     print(f'{ing}: {count}')   
    return sorted_dict

In [ ]:
def draw_ingridient_pie_chart(top_n):
    # print()
    ingredient_counts = get_ingridients_dict()

    # Extract the top N ingredients by frequency
    ingredient_names = collections.Counter(ingredient_counts).most_common(top_n)
    ingredient_names = [x[0] for x in ingredient_names]
    ingredient_frequencies = [ingredient_counts[name] for name in ingredient_names]
    total_receipe_count = sum(ingredient_frequencies)
    ingredient_percentages = [(count/total_receipe_count)*100 for count in ingredient_frequencies]
    ingredient_labels = ['{} ({:.1f}%)'.format(name, percentage) for name, percentage in zip(ingredient_names, ingredient_percentages)]

    # Create the pie chart
    plt.pie(ingredient_percentages, labels=ingredient_labels)
    plt.title('Top {} Ingredients'.format(top_n))
    plt.show()

looks like it works properly :)

now we want to see if there is a correlation between the ingridients and the rating to achive this we "exploded" our ingridients into binary columns 

In [ ]:
def explode_ingridients_and_get_corr(df):#this function expands the dataframe to each ingridient and tries to find a correlation
    #df = pd.read_csv('clean_modified.csv')

    # extract the ingredients column
    df_ing = df['Ingredients']

    # define a list of ingredients you care about
    important_ingredients = []

    with open('ingridients.csv', newline='') as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            important_ingredients.extend(row)

    # loop through the ingredients you care about
    for ingredient in important_ingredients:
        # create a new column with a value of 1 if the recipe contains the ingredient and 0 otherwise
        df[ingredient] = df_ing.str.contains(ingredient).astype(int)

    correlations = df[['Rating'] + [ingredient for ingredient in important_ingredients]].corr()

    # print the correlation coefficients for each ingredient
    # print(correlations.loc['Rating'])
    top_20 = correlations.loc['Rating'].sort_values(ascending=False)[1:21]
    print("Top 20 ingredients with the highest correlation with the 'Rating' column:")
    print(top_20)
    return df

In [ ]:
df= explode_ingridients_and_get_corr(df)

(fancy english word ) we found no correlations with ingridients and the rating (liquer was funny)
so that answears one of our research questions beucase nothing gave us a correlation with rating we consider that there isnt anything that can *directly* predict the rating of a recepie to further demonstrate this we decided to draw a heatmap and show that there is nothing special in it

the natural next step is to try machine learning and see if it can predict the rating

In [ ]:
#(get linear regression from shapira)

our accuracy hovers around 0.36(to confirm) which isnt that good so we played around with feature engineering to try and improve our predictions we decide to make a popularity colum that would sum the frequncy of each ingridient in the recepie also we decided to make a difficulty column that takes into acount the steps cooking time etc...(shapira plz fix)

In [ ]:
def add_popularity_score_to_df(df):
    binary_columns = [col for col in df.columns if col not in ['Name', 'Prep', 'Cook', 'Total', 'Servings', 'Rating', 'Rating_Count', 'Dairy', 'Meat', 'Fur', 'Calories', 'Fat', 'Carbs', 'Protein', 'Num_ing', 'Steps', 'Ingredients']]

    # Calculate the frequencies of each binary value
    ingredient_frequencies = df[binary_columns].sum() / len(df) * 30

    # Calculate the popularity score for each recipe
    popularity_scores = []
    for _, row in df.iterrows():
        score = 0
        for column in binary_columns:
            if row[column] == 1:
                score += ingredient_frequencies[column]
        popularity_scores.append(score)

    # Add the popularity scores as a new column to the DataFrame
    df['Popularity Score'] = popularity_scores

    # Print the updated DataFrame
    #print(df)
    #scraping_functions.draw_histo_1_params(df,'Popularity Score')
    return df

In [ ]:
def add_difficulty_column(df):
    # Read the CSV file into a pandas dataframe
    #df = pd.read_csv(csv_file_path)
    
    # Calculate the combined score for each recipe
    df['Combined'] = df['Num_ing'] + df['Steps'] + df['Total']
    
    # Calculate the mean and standard deviation of the combined score
    mean = df['Combined'].mean()
    std_dev = df['Combined'].std()
    
    # Calculate the z-score for each recipe
    df['Z_score'] = (df['Combined'] - mean) / std_dev
    
    # Divide the z-scores into 5 equal-sized groups
    df['Difficulty'] = pd.qcut(df['Z_score'], q=5, labels=[1, 2, 3, 4, 5])
    
    # Write the updated dataframe to a new CSV file
    # new_csv_file_path = csv_file_path.split('.csv')[0] + '_with_difficulty.csv'
    # df.to_csv(new_csv_file_path, index=False)
    #df.to_csv(csv_file_path, index=False, mode='w')
    #save_df(df,"csv_Wtih_diff.csv")
    
    # Print the number of recipes in each difficulty level
    for i in range(1, 6):
        count = df['Difficulty'][df['Difficulty'] == i].count()
        print(f"Difficulty level {i}: {count}")
    
    return df

lets re run our linear regression model 

In [ ]:
#predict_rating_linear(df) ## shapira i need your code boiiiii

amazingly we got a score of 0.54 which is bad but it is a 40% improvement over our previous attempt
obviously we didnt include columns that and feature engineering attempts that didnt have significant results 
among the ideas we tried (insert bullshit from chatgpt here shapira plz fix)

seeing as our result is 0.54 we decided to change tactics a little even though our question is Is it possible to determine if a recipe will receive a high rating? we dont have to predict it stars and go from there

instead what we decided is that a recepie which is rated for 5 stars is agruably a good recepie (we have debates where can you draw the line maybe 4.5 is also good enough but ulimately 5 stars is supposed to be the best of the best so we went with that)

we decided to eventually use a model called perseptron which is a binary classification model meaning its trained to figure out if a recepie is 5 stars or not( we were advised to go with this aproach by our instructor)

In [ ]:
from sklearn.base import accuracy_score
from sklearn.linear_model import Perceptron
from sklearn.model_selection import train_test_split


def build_perceptron_model(df):
    #df = pd.read_csv('test123.csv')

    # Save the 'Rating' column separately
    ratings = df['Rating']
    
    # Drop the 'Rating' column from the dataframe
    df = df.drop(['Rating'], axis=1)
    df = df.drop(['Rating_Count'], axis=1)
    df = df.select_dtypes(include=['int', 'float'])
    # Get the column names of the dataframe
    col_names = df.columns

    # Convert the dataframe to a numpy array
    X = df.select_dtypes(include=['int', 'float']).values

    # Define the target variable
    y = (ratings > 4.5).astype(int).values

    # Verify that X and y have the same length
    assert len(X) == len(y), "X and y must have the same length"

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Fit the Perceptron model to the training data
    model = Perceptron()
    model.fit(X_train, y_train)

    # Compute the predictions on the test data
    y_pred = model.predict(X_test)

    # Compute the accuracy of the model
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy: {accuracy:.2f}")

    # Get the weights learned by the perceptron
    weights = model.coef_[0]

    # Create a new dataframe with the column names and weights
    weights_df = pd.DataFrame({'column': col_names, 'weight': weights})

    # Print the weights dataframe
    print(weights_df)

    #save_df(weights_df,'FML.csv')
    return weights_df

In [ ]:
df = build_perceptron_model(df)

we got an amazing result of 0.95 which is good too good to be true so we tested on 20 recepies from another site (recepies that didnt apear on our site ) and it accurately predicted if they were 5 star recepies or not 

In [ ]:
sorted_df = df.sort_values(by='weight')

# printing the top 10 rows
print(sorted_df.tail(10)[['column', 'weight']])

# printing the bottom 10 rows
print(sorted_df.head(10)[['column', 'weight']])

we also printed the weights of the recepie the top 5 most contributing factors to a recepie being a 5 star recepie are (fix? )
Prep time
Cook time
Servings
Calories
Number of ingredients
Number of steps
as their weights were the highest in our weights dataframe also the 5 worst ingridients were
Popularity Score with a weight of -5960.882178301813
Prep with a weight of -2079.0
Cook with a weight of -5674.0
Total with a weight of -7753.0
Servings with a weight of -1836.0

these are high number that help define recepies that are not! 5* from this we can understand that people liked the recepies with shorter cooking time be it prep the cooking itself or the mix , low amount of servings(meaning high calories because of our previous results) and unique ingridients because the popularity score is highly negetive
which supports our idea of adding rare ingridients (caviar,lobster,etc) can improve the odds of making a recepie a high rating one :)

to conclude : bla bla bla shapira pls help :)

visualize results? haha updated